In [224]:
import streamlit as st
from streamlit_timeline import timeline
import json
import os
import io
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
from ydata_profiling import ProfileReport
from streamlit_pandas_profiling import st_profile_report
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
import joblib

In [200]:
df = pd.read_csv('stroke.csv', index_col=0)
display(df)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0
...,...,...,...,...,...,...,...,...,...,...,...
15299,Female,22.0,0,0,No,Govt_job,Urban,72.63,19.5,never smoked,0
15300,Female,46.0,1,0,Yes,Private,Urban,101.19,32.1,never smoked,0
15301,Female,75.0,0,0,Yes,Self-employed,Urban,87.69,26.2,never smoked,0


In [220]:
# select machine lernning model
def select_model(model):
    if model=="Logistic Regression":
        metric = accuracy_score
        ML_model = LogisticRegression()
    elif model == "Decision Tree":
        metric = accuracy_score
        ML_model = tree.DecisionTreeClassifier()
    elif model == "Random Forest":
        metric = accuracy_score
        ML_model = RandomForestClassifier()
    elif model == "Linear Regression":
        metric = mean_squared_error
        ML_model = LinearRegression()
    elif model == "Regression Tree":
        metric = mean_squared_error
        ML_model = tree.DecisionTreeRegressor()
    elif model == "Ridge Regression":
        metric = mean_squared_error
        ML_model = Ridge()

In [201]:
# Custom imputation using random values between min and max for each column
def random_impute(X):
    for i in range(X.shape[1]):
        col = X[:, i]
        missing = np.isnan(col)
        col_min, col_max = np.nanmin(col), np.nanmax(col)
        col[missing] = np.random.uniform(col_min, col_max, size=missing.sum())
    return X
        
def clean_data(X, imputation_strategy='most_frequent', scaling_method='minmax'):
    # Separate numerical and non-numerical columns
    numerical_cols = X.select_dtypes(include=[np.number]).columns
    categorical_cols = X.select_dtypes(exclude=[np.number]).columns

    # Handle missing values based on the chosen imputation strategy
    if imputation_strategy in ['mean', 'median','most_frequent']:
        imputer = SimpleImputer(strategy=imputation_strategy)
        X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    elif imputation_strategy == 'random':        
        X_imputed = random_impute(X.copy())  # Apply random imputation before scaling
    else:
        raise ValueError("Invalid imputation strategy. Choose 'mean', 'median', 'most_frequent' or 'random'.")
    
    # Encode non-numerical columns with integer encoding
    X_imputed[categorical_cols] = X_imputed[categorical_cols].apply(LabelEncoder().fit_transform)


    # Apply scaling based on the chosen scaling method
    if scaling_method == 'minmax':
        scaler = MinMaxScaler()
    elif scaling_method == 'standard':
        scaler = StandardScaler()
    else:
        raise ValueError("Invalid scaling method. Choose 'minmax' or 'standard'.")
    
    # Fit and transform the data
    X_scaled = scaler.fit_transform(X_imputed)
    X_cleaned = pd.DataFrame.from_records(data=X_scaled, columns=X.columns)
    return X_cleaned

In [202]:
def train_model(model, train_test_split, folds):
    X_train, X_test, y_train, y_test = train_test_split
    param_grid = model["params"]
    method = model["method"]
    metric = param_grid["criterion"]

    # Create a GridSearchCV object with cross-validation
    grid_search = GridSearchCV(method, param_grid, cv=folds, n_jobs=-1, scoring=metric, return_train_score=True)

    # Fit the model on the training data
    grid_search.fit(X_train, y_train)
    
    return grid_search.best_estimator_, grid_search.best_params_, grid_search.best_score_, grid_search.cv_results_

In [228]:
def ML_Pipeline(json_config):
    
    


    target = train_config["target"]
    folds = train_config["folds"]
    
    df = clean_data(data)
    X = df.drop(target, axis=1)
    y = df[target]
    
    data_split = train_test_split(X, y, test_size = train_config["test_size"], shuffle=True, random_state=42)
    
    stats = []
    hyperparams = []
    trained_models = []     

    for i,model in enumerate(models):               
        trained_model, best_params, val_score, cv_results= train_model(model, data_split,folds)
        
        s = {"validation_score": val_score, "cv_summary": pd.DataFrame(cv_results)}
        stats.append(s)
        trained_models.append(trained_model)
        hyperparams.append(best_params)
    return stats, trained_models, hyperparams

def get_best_model(stats, trained_models):
    index = np.argmin(stats[1,:])
    best_value = np.min(stats[1,:])
    model= trained_models[index]
    return model,best_value,index

In [233]:
with open("configuration_classification.json") as json_data:
    data = json.load(json_data)
print(data["Runs"])
print(data["Models"])
print(data["Training"])

{'name': 'RF', 'model_type': 'Decision Tree', 'params': {'nModels_search': 5, 'n_estimators': [100, 500], 'max_depth': [1, 20], 'min_samples_split': [2, 8], 'min_samples_leaf': [1, 4], 'max_features': ['auto', 'sqrt'], 'bootstrap': ['True', 'False'], 'criterion': ['gini', 'entropy', 'log_loss']}}
{'name': 'DT', 'model_type': 'Decision Tree', 'params': {'max_depth': [1], 'min_samples_split': [2], 'min_samples_leaf': [1], 'max_features': 'auto', 'max_leaf_nodes': [23], 'min_weight_fraction_leaf': [0.0], 'criterion': 'gini', 'splitter': 'best'}}
{'is_regression': False, 'CV': False, 'metric': None}


In [204]:
def save_model(model, model_name, folder_name='saved_models'):
    # Create the folder if it doesn't exist
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Define the file path
    file_path = os.path.join(folder_name, model_name + '.pkl')

    # Save the model using joblib
    joblib.dump(model, file_path)
    print(f"Model saved at: {file_path}")

In [216]:
param_grid_logreg = {
    'penalty': ['l1', 'l2'],    # Regularization type
    'C': [0.1, 1, 10],          # Inverse of regularization strength
    'solver': ['liblinear'],    # Optimization algorithm
    'l1_ratio': [0.5, 0.75, 1]          # For elasticnet, mix of L1 and L2 regularization
}

param_grid_rf = {
    'n_estimators': [100, 200],           # Number of trees in the forest
    'max_depth': [None, 10, 20],             # Maximum depth of each tree
    'min_samples_split': [2, 5],                 # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2],                     # Whether bootstrap samples are used when building trees
    'criterion': ['gini', 'entropy']     # Function to measure the quality of a split
}

param_grid_ridge = {
    'alpha': [0.1, 1, 10],   # Regularization strength
    'solver': ['auto', 'svd', 'cholesky'],  # Solver for optimization
    'fit_intercept': [True, False]
}


ridge = {"method": Ridge(), "params": param_grid_ridge, "metric": "mean_squared_error"}
rf = {"method": RandomForestClassifier(), "params": param_grid_rf, "metric": "accuracy"}
logreg = {"method": LogisticRegression(), "params": param_grid_logreg, "metric": "accuracy"}

train_settings = {"folds": 5, "target": "age", "test_size": 0.2}

stats, trained_models,hyperparams = ML_Pipeline(df, [ridge], train_settings)

In [219]:
# unpack stats
s = pd.DataFrame(stats[0]["cv_summary"])
print(s)
print(stats)

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  param_alpha  \
0        0.066471      0.029572         0.004461        0.000551          0.1   
1        0.992363      0.492570         0.005382        0.001542          0.1   
2        0.010878      0.002028         0.006450        0.004625          0.1   
3        0.010370      0.002298         0.004656        0.000724          0.1   
4        0.015847      0.002687         0.005822        0.001412          0.1   
5        0.011055      0.003327         0.004788        0.001251          0.1   
6        0.019468      0.013482         0.006193        0.003954          1.0   
7        0.018737      0.003918         0.007271        0.004548          1.0   
8        0.018134      0.006563         0.011436        0.006925          1.0   
9        0.009738      0.002305         0.005138        0.001349          1.0   
10       0.016764      0.006927         0.005335        0.001479          1.0   
11       0.009385      0.002

In [207]:
# SAMPLE USAGE
# train and test split
target = "stroke"
df = clean_data(df)
X = df.drop(target, axis=1)
y = df[target]
split = train_test_split(X, y, test_size = 0.2, shuffle=True, random_state=4)
X_train,X_test,y_train,y_test = split

In [208]:
# Evaluate the model on the test set
test_score = trained_models[0].score(X_test, y_test)
save_model(trained_models[0], "test")

Model saved at: saved_models\test.pkl


In [210]:
t = joblib.load("model_file_name.pkl")
ts =trained_models[0].score(X_test, y_test)
print(ts)

0.967984318850049
